In [1]:
# !wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
# !wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
# !wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# # Install the PyDrive wrapper & import libraries.
# # This only needs to be done once per notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
# downloaded.GetContentFile('cc.ru.300.vec.zip')

# downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
# downloaded.GetContentFile('cc.uk.300.vec.zip')

# !unzip cc.ru.300.vec.zip
# !unzip cc.uk.300.vec.zip

In [2]:
import numpy as np

from sklearn.linear_model import LinearRegression

from gensim.models import KeyedVectors

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [4]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [5]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

In [6]:
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [7]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923034667969),
 ('июль', 0.8234529495239258),
 ('август', 0.8120501041412354),
 ('декабрь', 0.803900420665741)]

In [8]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
      pred_list = ru_emb.most_similar(mapped_vectors[i:i+1])[:topn]
      pred_list = [j for j, _ in pred_list]
      num_matches += 1 if ru in pred_list else 0
    precision_val = num_matches / len(pairs)
    return precision_val

In [9]:
print(precision([("серпень", "август")], august, topn=5))
print(precision([("серпень", "август")], august, topn=9))
print(precision([("серпень", "август")], august, topn=10))

0.0
1.0
1.0


In [10]:
print(precision(uk_ru_test, mapping.predict(X_test)))
print(precision(uk_ru_test, mapping.predict(X_test), 5))

0.6356589147286822
0.813953488372093


### Улучшаем маппинг
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [11]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    XY_dot = np.dot(X_train.T, Y_train)
    u, _, vh = np.linalg.svd(XY_dot)
    W = np.dot(u, vh)
    return W

In [12]:
W = learn_transform(X_train, Y_train)
W.shape

(300, 300)

In [13]:
ru_emb.most_similar([np.matmul(uk_emb['серпень'], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025654554367065),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001736402511597),
 ('ноябрь', 0.7934483885765076),
 ('февраль', 0.7914121150970459),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [14]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6537467700258398
0.8242894056847545


In [15]:
with open("fairy_tale.txt", "r") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

uk_sentences[:5]

['лисичка - сестричка і вовк - панібрат',
 'як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !',
 'дай мені огня " .',
 'а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .',
 'прибігла на поле , аж там пасуть хлопці бичків .']

In [16]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    preds = []
    for word in sentence.split():
      try:
        word = mapping.predict(uk_emb[word].reshape(1, -1))
        word = ru_emb.most_similar(word)[0][0]
        preds.append(word)
      except KeyError:
        preds.append(word)

    return ' '.join(preds)

In [17]:
print(translate("."))
print(translate("1 , 3"))
print(translate("кіт зловив мишу"))

Во
от , до
кот поймал мышь


In [18]:
for sentence in uk_sentences:
    print(f"src: {sentence}\ndst: {translate(sentence)}\n")

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – девочка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , ой и пошла раз от однії бабы добувать огнь ; лечь во избу ой и говорит из " хороший день тебе , бабуля !

src: дай мені огня " .
dst: дай мне из " Во

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а баба только что вынуть со печи пирожок со маком , сладкий , ой и легла , чтобы он прохолов ; а лисичка коль и підгледала , ой притом что баба наклониться во печь , чтобы достать из , то лисичка сейчас ухватила пирожок ой и сожрать со избы , ой , бежать , весь мак со его виїла , а туда мусор на